In [176]:
import itertools as itools
import numpy as np

# Here I decided to implement a class 'Amplifier' which CONTAINS the IntcodeComputer function.
# This was done so that each amp could call its personal computer on its personal amplifier controller
# software (ACS), and it can then modify variables related to that particular amplifier (the phase, the state
# of the computer). Complete this description another time I'm going to sleep!



# Documentation for various amp_control_sws
def documentation():
    
    print("One or more user input are needed to run this Intcode.")
    print("Please run as: 'IntcodeComputer(Intcode, [USER_INPUT_1, USER_INPUT_2, ...])'")
    print("The software can also be run in 'feedback' mode, upon provising the explicit argument")
    print("in the function call as 'IntcodeComputer(Intcode, user_input, feedback = True).'")
    print("where USER_INPUT_i is an integer number. Refer to the text of the exercise")
    print("for how to run this computer.")
    #print(" - 1 for diagnostics")
    #print(" - 5 for Thermal Radiator Controller ")
    
    return 0


# A structured data type for instructions
class Instruction:
        
    def __init__(self, code):
        
        # Sets that associates an opcode with its associated number of parameters
        # 3 parameters opcodes
        III_params = [1, 2, 7, 8]
        # 2 parameters opcodes
        II_params = [5, 6]
        # 1 parameter opcodes
        I_params = [3, 4]
        
        # Variables of the instruction set
        # Increment value after instruction
        self.increment = 0
        # Number of parameters
        self.n_params = 0
        # Parameter modes
        self.param_mode = []
        # Control variable
        self.is_corrupted = False
        # Number of digits that make up the opcode
        opcode_digits = 2
        
        lscode = list(str(code))
        self.opcode = int(''.join(lscode[-opcode_digits:])) # This works even if len(o) = 1!
        
        if self.opcode in III_params:
            self.increment = 4
            self.n_params = 3
            # My favorite line <3
            #self.param_mode = [int(x) if len(o) > 2 else 0 for x in reversed(o[:-2])]
        elif self.opcode in I_params:
            self.increment = 2
            self.n_params = 1
            
        elif self.opcode in II_params:
            self.increment = 3
            self.n_params = 2
       
        elif self.opcode == 99:
            pass
            
        else:
            self.is_corrupted = True
                    
        for i in range(self.n_params):
            try:
                rlscode = lscode[::-1]    # Reversed 'lscode'
                self.param_mode.append(int(rlscode[opcode_digits + i]))
                
            except:
                self.param_mode.append(0)
         
    def ShowInstructionData(self, code):
        print("Instruction: ", code)
        print("Opcode: ", self.opcode)
        print("Increment: ", self.increment)
        print("Number of parameters: ", self.n_params)
        print("parameters mode: ", self.param_mode)
    
    
# It is useful to define a class in this case

class Amplifier:
    
    def __init__(self):
        
        # Phase information
        self.phase = 0
        # Amplifier Controller Software (ACS): it's the intcode series of instructions
        self.amp_control_sw = []
        # The state index keep track of runtime information for ACS: it stores the position
        # of the next instruction in the ACS
        self.state_index = 0
        # When the ACS is completed, this variable indicates that the signal can be send
        # to the thrusters
        self.to_thrust = False
        
        return
        
    def load_amp_control_sw(self, Intcode):
        
        self.amp_control_sw = [item for item in Intcode]
        
        return
        
    def set_phase(self, ph):
        
        self.phase = ph

        return
    
    # Set the state index of the amplifier
    def set_index(self, index):
        
        self.state_index = index
    
    # Reset all information of the amplifier to their initial value
    def reset(self):
        
        self.phase = 0
        self.amp_control_sw = []
        self.state_index = 0
        self.to_thrust = False
        
    # The Intcode computer
    def IntcodeComputer(self, input_signal):
    
        curr_pos = self.state_index
            
        i = Instruction(self.amp_control_sw[curr_pos])
    
        while(i.opcode != 99): 
        
            if not i.is_corrupted:
            
                # Initialize variables if needed for execution of the instruction
                if i.n_params > 0:
                    a = self.amp_control_sw[curr_pos+1] if i.param_mode[0] else self.amp_control_sw[self.amp_control_sw[curr_pos+1]]
                if i.n_params >= 2:
                    b = self.amp_control_sw[curr_pos+2] if i.param_mode[1] else self.amp_control_sw[self.amp_control_sw[curr_pos+2]]
   
                # Add
                if i.opcode == 1:
                    if i.param_mode[2]:
                        self.amp_control_sw[curr_pos+3] = a+b
                    else:
                        self.amp_control_sw[self.amp_control_sw[curr_pos+3]] = a+b
                    
                    curr_pos += i.increment
            
                # Multiply
                elif i.opcode == 2:
                    if i.param_mode[2]:
                        self.amp_control_sw[curr_pos+3] = a*b
                    else:
                        self.amp_control_sw[self.amp_control_sw[curr_pos+3]] = a*b
                    
                    curr_pos += i.increment
                
            # Write input to address    
                elif i.opcode == 3:
                    
                    # If this is the first time running the software, provide phase data as input
                    if curr_pos == 0:
                        self.amp_control_sw[self.amp_control_sw[curr_pos+1]] = self.phase
                    
                    # else provide input signal given as input
                    else:
                        self.amp_control_sw[self.amp_control_sw[curr_pos+1]] = input_signal
                    
                    curr_pos += i.increment
                
            
                # Print output from address
                elif i.opcode == 4:
                    output = self.amp_control_sw[curr_pos+1] if i.param_mode[0] else self.amp_control_sw[self.amp_control_sw[curr_pos+1]]
                
                    #print("Output: ", output, '\n')
                    curr_pos += i.increment
                    
                    # Save the current state for the future: the next time the software is run on the amplifier
                    # it wll start from this instruction
                    self.state_index = curr_pos
                       
                    # Halt the program and return output
                    return output
             
                # jump-if-True
                elif i.opcode == 5:
                    if a != 0:
                        curr_pos = b
                    else:
                        curr_pos += i.increment
            
                # jump-if-false
                elif i.opcode == 6:
                    if a == 0:
                        curr_pos = b
                    else:
                        curr_pos += i.increment
                
                # less-than
                elif i.opcode == 7:
                    if i.param_mode[2]:
                        self.amp_control_sw[curr_pos+3] = 1 if a < b else 0
                    else:
                        self.amp_control_sw[self.amp_control_sw[curr_pos+3]] = 1 if a < b else 0
                    
                    curr_pos += i.increment
            
                # equal
                elif i.opcode == 8:
                    if i.param_mode[2]:
                        self.amp_control_sw[curr_pos+3] = 1 if a == b else 0
                    else:
                        self.amp_control_sw[self.amp_control_sw[curr_pos+3]] = 1 if a == b else 0
                    
                    curr_pos += i.increment
        
            # If opcode is unknown
            else:
                print("Unknown opcode '" + str(i.opcode) + "'. Terminating.")
                return -1
   
            # Read the future instruction
            i = Instruction(self.amp_control_sw[curr_pos])
     
        
        # The program has come to the natural halting: reset index variable to beginning and 
        # proceed to pass the signal to thrusters (or the next amp in the circuit)
        self.state_index = 0
        self.to_thrust = True
        
        return 0


In [180]:
# Load input data from file
with open("input.txt", 'r') as infile:
    strIntcode = infile.read().split(',')
    
Intcode = np.array([int(x) for x in strIntcode])

In [181]:
# Initialize the 5 amplifier and the circuit for convenience
amp_A = Amplifier()
amp_B = Amplifier()
amp_C = Amplifier()
amp_D = Amplifier()
amp_E = Amplifier()

circuit = [amp_A, amp_B, amp_C, amp_D, amp_E]


In [182]:
max_output = 0
phase_seq_at_max = []

# Cycle over permutations of [0,1,2,3,4]
for it in itools.permutations(range(5, 10)):
    
    # Reset input signal
    input_signal = 0
    output_signal = 0
    
    # Reset amplifier to initial state and load Amp Control Software
    n = 0
    for amp in circuit:
        amp.reset()
        amp.load_amp_control_sw(Intcode)
        amp.set_phase(it[n])
        n+=1
        
    
    while amp_E.to_thrust != True:
        
        final_output = output_signal
        
        output_signal = amp_A.IntcodeComputer(input_signal)
        input_signal = output_signal
        
        output_signal = amp_B.IntcodeComputer(input_signal)
        input_signal = output_signal
        
        output_signal = amp_C.IntcodeComputer(input_signal)
        input_signal = output_signal
        
        output_signal = amp_D.IntcodeComputer(input_signal)
        input_signal = output_signal
        
        output_signal = amp_E.IntcodeComputer(input_signal)
        input_signal = output_signal
        
    # Look for max value
    if final_output > max_output:
        max_output = final_output
        phase_seq_at_max = it
                

print(max_output, phase_seq_at_max)

63103596 (8, 5, 9, 7, 6)
